## astropy.wcs
Implements the FITS WCS standard and some commonly used distortion conventions.

This tutorial will show how to create a WCS object from a FITS file and how to use it to transform coordinates.

In [ ]:
import os
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy import wcs

## Creating a WCS object from the header of a FITS file

Open a file with `astropy.fits` and look at it. (This file contains only the header and no data)

In [ ]:
sip_file_name = get_pkg_data_filename('data/sip.fits', package='astropy.wcs.tests')

sip_file = fits.open(sip_file_name)
sip_file.info()

To create a WCS object pass the header with the WCS keywords to astropy.wcs.WCS. In this case it is the primary header. A warning is issued due to the lack of image data. 

In [ ]:
w = wcs.WCS(sip_file[0].header)

## Transforming between pixel coordinates and sky coordinates

To perform the transformation from detector to sky, including distortions, pass x and y and an 'origin'. The third argument, 'origin', indicates whether the coordinates are 1-based (like FITS), or 0-based (like python).

The inputs can be numbers, numpy arrays or array like objects.

In [ ]:
ra, dec = w.all_pix2world([1, 100], [2, 200], 1)
print(ra, dec)

Perfom the inverse transformation - from sky to detector coordinates.

If analytical inverse is not available (often the case in the presence of distortion), then an iterative inverse is performed.

In [ ]:
print(w.all_world2pix(ra, dec, 1))

In some cases it is useful to omit the distortion and perform the core WCS transforms only:

In [ ]:
ra, dec = w.wcs_pix2world([1, 100], [2, 200], 1)
print(ra, dec)

In [ ]:
w.wcs_world2pix(ra, dec, 1)

## Creating a WCS programatically

A WCS object can be created programatically. Here is a concise example with 1 arcsecond pixels that is aligned with "North up, East to the left".

In [ ]:
my_wcs = wcs.WCS(naxis=2)
my_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']
my_wcs.wcs.crpix = [512, 512]
my_wcs.wcs.crval = [70., 20.]
my_wcs.wcs.cdelt = [-1/3600, 1/3600]
my_wcs.array_shape = [1024, 1024] # NAXIS2, NAXIS1
my_wcs

## Converting a WCS into an astropy.io.fits.Header instance

The WCS object can be changed and the new WCS can be written out as a new header.

By default only the primary WCS keywords are written out to the header. Pass a keyword `relax=True` to write out the SIP distortion.

In [ ]:
# The original WCS
w.printwcs()

In [ ]:
w.wcs.crpix = [200, 200]

# Calling *to_header()* without arguments writes
# out the standard WCS keywords.
w.to_header()

In [ ]:
# Passing *relax=True* writes out the SIP coefficients
# and all other distortions.
w.to_header(relax=True)

## Exercise: Refine a WCS using a list of detections and a reference catalog

Refine a WCS for a science image exposure from the Zwicky Transient Facility from these ingredients:
* An initial header
* A detection list cut at 17th magnitude, in file `data/ztf_detections_17thmag.csv`
* A reference catalog with coordinates and magnitudes from Gaia cut at 17 Gaia G magnitude, in `data/Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv`

The exercise makes use of `astropy.wcs`, `astropy.coordinates` and the projection capabilities of WCSAxes.

1. Read in the detection list and the reference catalog with `astropy.table.Table.read`
2. Calculate starting RAs and Decs for the detection list using the initial WCS
3. Create SkyCoord instances for the initial detection coordinates and the Gaia coordinates
4. Plot the detection list and the Gaia list in a scatter plot
5. Match the detection list and the Gaia list
6. Refine the WCS using the `fit_wcs_from_points` function from `astropy.wcs.utils`

Import everything we'll need for the exercise.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.wcs import WCS
from astropy.wcs.utils import fit_wcs_from_points
import astropy.units as u

%matplotlib inline

Create the initial WCS programatically.

In [ ]:
initial_wcs = WCS(naxis=2)
initial_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']  
initial_wcs.wcs.crval = [149.07662386535503, 33.32164150821777]  
initial_wcs.wcs.crpix = [-3305.678, -7136.481]
initial_wcs.wcs.cd = [[-0.0002817188, -1.554e-07],
                      [-1.998e-07, -0.0002819204]]  
initial_wcs.array_shape = [3080, 3072] # NAXIS2, NAXIS1

In [ ]:
initial_wcs

### 1. Read in the detection list and the reference catalog

Read in the detections and the reference catalog using `astropy.table.Table` with `format='csv'`.
The detections table is in `'data/ztf_detections_17thmag.csv'` and the reference catalog is `'data/Gaia-gaia_dr2_source-ztf-20190606224213_000667_zr.csv'`

In [ ]:
#detections = Table.read(...)

In [ ]:
#ref_catalog = Table.read(...)

### 2. Calculate starting RAs and Decs for the detection list using the initial WCS

Use the `initial_wcs.all_pix2world` function to calculate starting RA and Dec from the `detections['xpos']` and `detections['ypos']` columns. The pixel positions use the FITS numbering convention.

In [ ]:
#initial_ra, initial_dec = initial_wcs.all_pix2world(...)

### 3. Create SkyCoord instances for the initial detection coordinates and the Gaia coordinates

In [ ]:
#initial_coords = SkyCoord(...)

In [ ]:
#gaia_coords = SkyCoord(...)

### 4. Plot the detection list and the Gaia list in a scatter plot

Use `projection=initial_wcs` to make a scatter plot using `gaia_coords` and `initial_coords`. The open circles are sized according to magnitude.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.subplot(projection=initial_wcs)

# Uncomment this block to plot the Gaia coordinates
#ax.scatter(gaia_coords.ra, 
#           gaia_coords.dec,  c=None, marker='o',
#           s=20*(18 - ref_catalog['phot_g_mean_mag']),
#           facecolors='none', edgecolors='green',
#           transform=ax.get_transform('world'))

# Repeat for `initial_coords` but use `edgecolors='blue'

### 5. Match the detection list and the Gaia list

Use the `initial_coords.search_around_sky` method with a 15 arcsecond radius.

In [ ]:
#idxgaia, idxdet, d2d, d3d = initial_coords.search_around_sky(...)

In [ ]:
#gaia_matched = gaia_coords[idxgaia]
#detections_xpos_matched = detections['xpos'][idxdet]
#detections_ypos_matched = detections['ypos'][idxdet]
#print(len(gaia_matched), len(detections_xpos_matched), len(detections_ypos_matched))

### 6. Refine the WCS using the `fit_wcs_from_points` function

Look at the help for `fit_wcs_from_points` and use it to fit a new WCS. Use `sip_degree=3` to fit 3rd-order polynomial distortion.

Optionally, calculate new RAs and Decs for the matched pixel coordinates, and make another scatter plot.

In [ ]:
fit_wcs_from_points?

In [ ]:
#fitted_wcs = fit_wcs_from_points(...)

In [ ]:
#fitted_wcs

Examine the SIP distortion coefficients

In [ ]:
#fitted_wcs.sip.a

In [ ]:
#fitted_wcs.sip.b

Optionally, calculate new RAs and Decs for the matched pixel coordinates, and make another scatter plot.

### APPENDIX: Optional material on TPV distortion

The TPV distortion convention understood by SExtractor and related programs is also understood by `astropy.wcs`.

A header can be read from a text file and used to create the WCS. Here we use a header from an
image from the Zwicky Transient Facility, a time-domain imaging survey.

In [ ]:
tpv_header = fits.Header.fromtextfile(os.path.join('data', 'ztf_sci_header.txt'))

In [ ]:
w_tpv = wcs.WCS(tpv_header)

The keywords starting with `PV` specify the distortion

In [ ]:
w_tpv.to_header()

In the TPV case, `all_pix2world` gives the same result as `wcs_pix2world`.

This is because the underlying WCSLIB library is handling the distortion.

In [ ]:
print(w_tpv.all_pix2world([[500,1000]], 1))

In [ ]:
print(w_tpv.wcs_pix2world([[500,1000]], 1))